# 自動機械学習 Automated Machine Learning による離反予測モデリング & モデル解釈

顧客属性データを用いて離反予測モデルを開発します。
- Python SDK のインポート
- Azure ML service Workspace への接続
- Experiment の作成
- データの準備
- 自動機械学習の事前設定
- モデル学習と結果の確認
- モデル解釈

## 1. 事前準備
### Python SDK のインポート
Azure Machine Learning service の Python SDKをインポートします

In [1]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

W0919 00:14:02.221629 4581463488 deprecation_wrapper.py:119] From /Users/konabuta/miniconda3/envs/myenv/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/common/tf_wrappers.py:36: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0919 00:14:02.222572 4581463488 deprecation_wrapper.py:119] From /Users/konabuta/miniconda3/envs/myenv/lib/python3.6/site-packages/azureml/automl/core/_vendor/automl/client/core/common/tf_wrappers.py:36: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



### Azure ML workspace との接続
Azure Machine Learning service との接続を行います。Azure に対する認証が必要です。

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

azureml	eastus	mlservice	eastus


### 実験名の設定

In [3]:
# choose a name for experiment
experiment_name = 'automl-classification-churn'
experiment=Experiment(ws, experiment_name)

### 学習データの準備

In [4]:
data = "https://amlgitsamples.blob.core.windows.net/churn/CATelcoCustomerChurnTrainingSample.csv"
df = Dataset.Tabular.from_delimited_files(data).to_pandas_dataframe()

In [5]:
df.head()

,age,annualincome,calldroprate,callfailurerate,callingnum,customerid,customersuspended,education,gender,homeowner,...,totalminsusedinlastmonth,unpaidbalance,usesinternetservice,usesvoiceservice,percentagecalloutsidenetwork,totalcallduration,avgcallduration,churn,year,month
0,12,168147,0.06,0.00,4251078442,1,True,Bachelor or equivalent,Male,True,...,15,19,False,False,0.82,5971,663,0,2015,1
1,12,168147,0.06,0.00,4251078442,1,True,Bachelor or equivalent,Male,True,...,15,19,False,False,0.82,3981,995,0,2015,2
2,42,29047,0.05,0.01,4251043419,2,True,Bachelor or equivalent,Female,True,...,212,34,False,True,0.27,7379,737,0,2015,1
3,42,29047,0.05,0.01,4251043419,2,True,Bachelor or equivalent,Female,True,...,212,34,False,True,0.27,1729,432,0,2015,2
4,58,27076,0.07,0.02,4251055773,3,True,Master or equivalent,Female,True,...,216,144,False,False,0.48,3122,624,0,2015,1


In [6]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["churn"],axis=1)
y = df["churn"].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=100,stratify=y)

## 2. 自動機械学習 Automated Machine Learning
### 学習事前設定

In [7]:
automl_settings = {
    "iteration_timeout_minutes": 5,
    "iterations": 5,
    "n_cross_validations": 2,
    "primary_metric": 'AUC_weighted',
    "preprocess": True,
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'classification',
                             X = X_train,
                             y = y_train,
                             **automl_settings
                            )

### 実行と結果確認

In [8]:
local_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_0e7b0d94-8674-4227-a47b-61dd2a4843c5
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Missing Values Imputation
STATUS:       PASSED
DESCRIPTION:  There were no missing values found in the training data.

TYPE:         High Cardinality Feature Detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.

**************************************************************

In [9]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
# 詳細ログの出力
local_run.get_details()

{'runId': 'AutoML_0e7b0d94-8674-4227-a47b-61dd2a4843c5',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2019-09-18T15:15:43.020287Z',
 'endTimeUtc': '2019-09-18T15:17:59.529736Z',
 'properties': {'num_iterations': '5',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'MaxTimeSeconds': '300',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'RawAMLSettingsString': "{'name': 'automl-classification-churn', 'path': '.', 'subscription_id': '9c0f91b8-eb2f-484c-979c-15848c098a6b', 'resource_group': 'mlservice', 'workspace_name': 'azureml', 'region': 'eastus', 'compute_target': 'local', 'spark_service': None, 'iterations': 5, 'primary_metric': 'AUC_weighted', 'task_type': 'classification', 'data_script': None, 'validation_size': 0.0, 'n_cross_validations': 2, 'y_min': None, 'y_max': None, 'num_classes': 2, 'preprocess': True, 'lag_length': 0, 'is_timeseries': False, 'm

In [11]:
best_run, fitted_model = local_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-classification-churn,AutoML_0e7b0d94-8674-4227-a47b-61dd2a4843c5_2,,Completed,Link to Azure Portal,Link to Documentation


### モデルの理解

In [12]:
fitted_model.named_steps['datatransformer'].get_featurization_summary()

[{'RawFeatureName': 'age',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'annualincome',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'calldroprate',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'callfailurerate',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'callingnum',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'customerid',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'monthlybilledamount',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
 

In [13]:
from pprint import pprint


def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()


print_model(fitted_model)

datatransformer
{'enable_feature_sweeping': None,
 'feature_sweeping_timeout': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None}

StandardScalerWrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing.data',
 'with_mean': False,
 'with_std': False}

LightGBMClassifier
{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 0.6933333333333332,
 'importance_type': 'split',
 'learning_rate': 0.07894947368421053,
 'max_bin': 240,
 'max_depth': 3,
 'min_child_samples': 318,
 'min_child_weight': 6,
 'min_split_gain': 0.631578947368421,
 'n_estimators': 50,
 'n_jobs': 1,
 'num_leaves': 65,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.5789473684210527,
 'reg_lambda': 0.631578947368421,
 'silent': True,
 'subsample': 0.3963157894736842,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'verbose': -10}



## 3. Automated ML Explainer

In [14]:
from azureml.train.automl.automl_explain_utilities import AutoMLExplainerSetupClass, automl_setup_model_explanations

automl_explainer_setup_obj = automl_setup_model_explanations(fitted_model, X=X_train, X_test=X_test, y=y_train, task='classification')

Current status: Setting up data for AutoMl explanations
Current status: Setting up the AutoML featurization for explanations
Current status: Setting up the AutoML estimator
Current status: Setting up the AutoML featurizer
Current status: Generating a feature map for raw feature importance
Current status: Finding all classes from the dataset
Current status: Data for AutoMl explanations successfully setup


In [15]:
from azureml.explain.model.mimic.models.lightgbm_model import LGBMExplainableModel
from azureml.explain.model.mimic_wrapper import MimicWrapper
explainer = MimicWrapper(ws, automl_explainer_setup_obj.automl_estimator, LGBMExplainableModel, 
                         init_dataset=automl_explainer_setup_obj.X_transform, run=best_run,
                         features=automl_explainer_setup_obj.engineered_feature_names, 
                         feature_maps=[automl_explainer_setup_obj.feature_map],
                         classes=automl_explainer_setup_obj.classes)

Using older than supported version of lightgbm, please upgrade to version greater than 2.2.1


In [16]:
raw_explanations = explainer.explain(['local', 'global'], get_raw=True, 
                                     raw_feature_names=automl_explainer_setup_obj.raw_feature_names,
                                     eval_dataset=automl_explainer_setup_obj.X_test_transform)
#print(raw_explanations.get_feature_importance_dict())

In [17]:
from azureml.contrib.explain.model.visualize import ExplanationDashboard
ExplanationDashboard(raw_explanations, automl_explainer_setup_obj.automl_pipeline, automl_explainer_setup_obj.X_test_raw)

ExplanationWidget(value={'predictedY': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…